In [2]:
from torchvision import datasets,transforms
from torch import nn 
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

In [3]:
train_set = datasets.MNIST('mnist/',train=True,download=True,transform=transforms.ToTensor())
test_set = datasets.MNIST('mnist/',train=False,download=True,transform=transforms.ToTensor())

9913344it [00:05, 1754245.31it/s]                             


Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw



29696it [00:00, 421608.29it/s]           


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw



1649664it [00:01, 1320610.28it/s]                             


Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw



5120it [00:00, 15471784.21it/s]         

Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw



In [11]:
class LSTM(nn.Module):
    def __init__(self,input_dim,output_dim,hidden_dim,n_layers,batch_size):
        super(LSTM,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.batch_size = batch_size
        self.lstm = nn.LSTM(self.input_dim,self.hidden_dim,self.n_layers,batch_first=True)
        self.fc = nn.Linear(self.hidden_dim,self.output_dim)

    def forward(self,x):
        c0 = torch.zeros(self.n_layers,self.batch_size,self.hidden_dim).requires_grad_()
        h0 = torch.zeros(self.n_layers,self.batch_size,self.hidden_dim).requires_grad_()
        out,(h_t,c_t) = self.lstm(x,(h0,c0))
        output = F.relu(h_t[-1])
        return self.fc(output)


In [13]:
batch_size = 100
in_dim = 28
out_dim = 10
n_seq = 28
hidden_dim = 128
n_layers = 1
epochs = 5
losses = []
accs = []
model = LSTM(in_dim,out_dim,hidden_dim,n_layers,batch_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.003)
trainload = DataLoader(train_set,batch_size=batch_size)
testload = DataLoader(test_set,batch_size=batch_size)
for e in range(epochs):
    loss_tr = []
    acc_tr = []
    print(f'running [{e+1}/{epochs}]')

    model.train()
    for i, (img_tr,y_tr) in enumerate(iter(trainload)):
        #preventing grad. from accumulation
        optimizer.zero_grad()
        #reshaping images N X 1 X 28 X 28 -> N x n_Seq x num_features
        img_tr = img_tr.view(-1,n_seq,in_dim)
        output = model.forward(img_tr)
        loss = criterion(output,y_tr)
        loss.backward()
        optimizer.step()
        loss_tr.append(loss.item())
        acc_tr.append(y_tr.eq(output.detach().argmax(dim=1)).float().mean().item())
    losses.append(np.mean(loss_tr))
    accs.append(np.mean(acc_tr))
    print(f'\ttrain loss is : {losses[-1]:.4f}\taccuracy : {accs[-1]}')




running [1/5]
	train loss is : 0.5024	accuracy : 0.8411000014469028
running [2/5]
	train loss is : 0.1396	accuracy : 0.9595333378513654
running [3/5]
	train loss is : 0.0904	accuracy : 0.9739333421985308
running [4/5]
	train loss is : 0.0711	accuracy : 0.9795000101129214
running [5/5]
	train loss is : 0.0593	accuracy : 0.9823500100771586


In [18]:
testload = DataLoader(test_set,batch_size=batch_size)


loss_test = []
acc_test = []
with torch.no_grad():
    model.eval()
    for i, (img_test,y_test) in enumerate(iter(testload)):
        #preventing grad. from accumulation
        #reshaping images N X 1 X 28 X 28 -> N x n_Seq x num_features
        img_test = img_test.view(-1,n_seq,in_dim)
        output_test = model.forward(img_test)
        loss = criterion(output_test,y_test)
        loss_test.append(loss.item())
        acc_test.append(y_test.eq(output_test.detach().argmax(dim=1)).float().mean().item())

    print(f'\ttest loss : {np.mean(loss_test)}\taccuracy : {np.mean(acc_test)}')
    


	test loss : 0.07616689075483009	accuracy : 0.9782000064849854
